In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=1000)
tf.enable_eager_execution()
print(tf.__version__)

In [ ]:
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

In [ ]:
X, y = mnist.train.next_batch(1)
X

In [ ]:
X.shape

In [ ]:
y

In [ ]:
X.reshape((28, 28))

In [ ]:
plt.imshow(X.reshape(28, 28), cmap='Greys')
plt.show()

In [ ]:
X = X.reshape(1, 28, 28, 1)
X.shape

In [ ]:
y

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

In [ ]:
w1 = tf.Variable(initializer([3, 3, 1, 32]))
w1

In [ ]:
tf.reshape(w1[ : , : , : , 0], (3, 3))

In [ ]:
img = X

In [ ]:
img

In [ ]:
convolution_h1 = tf.nn.conv2d(X, w1, strides=1, padding='SAME')

In [ ]:
convolution_h1[0, : , : , 0]

In [ ]:
plt.imshow(convolution_h1[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
convolution_h = tf.nn.relu(convolution_h1)

In [ ]:
convolution_h1[0, : , : , 0]

In [ ]:
plt.imshow(convolution_h[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
convolution_h1 = tf.nn.max_pool(convolution_h1, ksize=2, strides=1, padding='SAME')
convolution_h1

In [ ]:
plt.imshow(convolution_h1[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
convolution_h1 = tf.reshape(convolution_h1, [-1, 28 * 28 * 32])
convolution_h1

In [ ]:
convolution_h1.shape

In [ ]:
# w3 = initializer([픽셀*이미지수, 회귀횟수])
w3 = tf.Variable(initializer([28 * 28 * 32, 10]))
w3

In [ ]:
y_h1 = tf.matmul(convolution_h1, w3)
y_h1

In [ ]:
y

In [ ]:
y_h1 = tf.nn.softmax(y_h1)
y_h1

In [ ]:
# mnist의 train data 갯수
mnist.train.num_examples

In [ ]:
# 학습시킬 image_size
batch_size = 100

total_batch = int(mnist.train.num_examples / batch_size)
print('total_batch : ', total_batch)

In [ ]:
optimizer = tf.train.AdamOptimizer(0.001)

In [ ]:
for i in range(total_batch) :
	print('i = ', i)

	# mnist.train.next_batch(batch_size) : 학습이미지에서 순서대로 batch_size개씩 반복해서 반환
	X, y = mnist.train.next_batch(batch_size)
	X = X.reshape(-1, 28, 28, 1)
	# print('X = ', X)
	# print('y = ', y)
	# print('=' * 100)

	with tf.GradientTape() as tape :
		# convolution 계산
		convolution_h1 = tf.nn.conv2d(X, w1, strides=1, padding='SAME')
		# relu 계산
		convolution_h1 = tf.nn.relu(convolution_h1)
		# max_pool 계산
		convolution_h1 = tf.nn.max_pool(convolution_h1, ksize=2, strides=1, padding='SAME')
		# 선형회귀를 위해 형변환
		convolution_h1 = tf.reshape(convolution_h1, [-1, 28 * 28 * 32])
		
		# 선형회귀
		y_h1 = tf.matmul(convolution_h1, w3)
		# softmax
		y_h1 = tf.nn.softmax(y_h1)

		# 오차(cost) 계산
		cost = -tf.reduce_sum(y * tf.log(y_h1)) / len(y_h1)

		# 기울기 계산
		grads = tape.gradient(cost, [w1, w3])

	# cost가 0이 되도록 w1, w3 수정
	optimizer.apply_gradients(grads_and_vars=zip(grads, [w1, w3]))
	# total_cost += cost

	print('cost = ', cost)
	print('=' * 100)